### Imports

In [12]:
import pandas as pd
import numpy as np

import qgrid
from IPython.display import display

import jupyterthemes.stylefx as jstyle
import jupyterthemes.jtplot as jplot
import jupyterthemes as jt

jstyle.toggle_settings(toolbar=True, nbname=True)
# jstyle.set_nb_theme('monokai')

### Reading in the data, checking for nulls or unexpected data types.

In [13]:
df = pd.read_csv("../data/device_failure_data_scientist.csv")
df.head()

,date,device,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9,failure
0,15001,S1F01085,215630672,56,0,52,6,407438,0,0,7,0
1,15001,S1F0166B,61370680,0,3,0,6,403174,0,0,0,0
2,15001,S1F01E6Y,173295968,0,0,0,12,237394,0,0,0,0
3,15001,S1F01JE0,79694024,0,0,0,6,410186,0,0,0,0
4,15001,S1F01R2B,135970480,0,0,0,15,313173,0,0,3,0


In [35]:
df.sort_values(by='device', inplace=True)
grid = qgrid.QGridWidget(df=df, show_toolbar=True)

In [36]:
grid

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [37]:
df.tail()

,date,device,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9,failure
30726,15039,Z1F2PBHX,233634680,0,0,0,5,157415,0,0,0,0
28591,15036,Z1F2PBHX,103204112,0,0,0,5,157415,0,0,0,0
54650,15074,Z1F2PBHX,198385440,0,0,0,5,161014,0,0,0,0
43187,15057,Z1F2PBHX,170476880,0,0,0,5,161003,0,0,0,0
45203,15060,Z1F2PBHX,208890824,0,0,0,5,161010,0,0,0,0


Checking for null values

In [38]:
df.isnull().sum()

date          0
device        0
attribute1    0
attribute2    0
attribute3    0
attribute4    0
attribute5    0
attribute6    0
attribute7    0
attribute8    0
attribute9    0
failure       0
dtype: int64

In [39]:
# Make sure all the devices are actually there and not empty strings
df['device'].sort_values(ascending=False)

45203    Z1F2PBHX
46547    Z1F2PBHX
36420    Z1F2PBHX
67274    Z1F2PBHX
60676    Z1F2PBHX
           ...   
2326     S1F01085
4651     S1F01085
5812     S1F01085
3489     S1F01085
0        S1F01085
Name: device, Length: 124494, dtype: object

In [40]:
df.shape

(124494, 12)

In [41]:
df.dtypes

date           int64
device        object
attribute1     int64
attribute2     int64
attribute3     int64
attribute4     int64
attribute5     int64
attribute6     int64
attribute7     int64
attribute8     int64
attribute9     int64
failure        int64
dtype: object

### Looking at the two classes
Baseline accuracy $= 0.999$ (very large class imbalance)

In [42]:
df['failure'].value_counts()

0    124388
1       106
Name: failure, dtype: int64

In [43]:
df['device'].unique()

array(['S1F01085', 'S1F013BB', 'S1F0166B', ..., 'Z1F26YZB', 'Z1F282ZV',
       'Z1F2PBHX'], dtype=object)

In [44]:
df['device'].value_counts()

S1F0E9EP    304
W1F05X69    304
W1F0JXDL    304
W1F0FZPA    304
Z1F0GB8A    304
           ... 
W1F1N4CT      5
Z1F0LCDA      5
W1F1CHZK      5
S1F04KSC      4
W1F0WJFT      3
Name: device, Length: 1168, dtype: int64

`failed_devices` is a list of all the devices that failed.  Failed devices had telemetry data until the day they did 
fail while not-failed devices (unsurprisingly) continued transmitting telemetry data.  I can use this list to filter 
the data frame to look specifically at devices that failed or devices that didn't fail, removing all of the data of 
failed devices.  I can better compare the differences in telemetry data and figure out which attributes are relevant.

In [45]:
failed_devices = list(df.query("failure == 1")['device'])
failed_devices

['S1F023H2',
 'S1F03YZM',
 'S1F09DZQ',
 'S1F0CTDN',
 'S1F0DSTY',
 'S1F0F4EB',
 'S1F0GG8X',
 'S1F0GJW3',
 'S1F0GKFX',
 'S1F0GKL6',
 'S1F0GPFZ',
 'S1F0GSD9',
 'S1F0GSHB',
 'S1F0J5JH',
 'S1F0JD7P',
 'S1F0JGJV',
 'S1F0L0DW',
 'S1F0LCTV',
 'S1F0LCVC',
 'S1F0LD15',
 'S1F0LD2C',
 'S1F0P3G2',
 'S1F0PJJW',
 'S1F0QF3R',
 'S1F0QY11',
 'S1F0RR35',
 'S1F0RRB1',
 'S1F0RSZP',
 'S1F0S2WJ',
 'S1F0S4CA',
 'S1F0S4EG',
 'S1F0S4T6',
 'S1F0S57T',
 'S1F0S65X',
 'S1F0T2LA',
 'S1F0TQCV',
 'S1F10E6M',
 'S1F11MB0',
 'S1F13589',
 'S1F135TN',
 'S1F136J0',
 'S1F13H80',
 'W1F03D4L',
 'W1F03DP4',
 'W1F08EDA',
 'W1F0F6BN',
 'W1F0FKWW',
 'W1F0FW0S',
 'W1F0GCAZ',
 'W1F0KCP2',
 'W1F0M35B',
 'W1F0M4BZ',
 'W1F0NZZZ',
 'W1F0P114',
 'W1F0PAXH',
 'W1F0PNA5',
 'W1F0Q8FH',
 'W1F0SGHR',
 'W1F0T034',
 'W1F0T074',
 'W1F0T0B1',
 'W1F0TA59',
 'W1F0VDH2',
 'W1F0WBTM',
 'W1F0X4FC',
 'W1F0X5GW',
 'W1F0Z1W9',
 'W1F0Z3KR',
 'W1F0Z4EA',
 'W1F11ZG9',
 'W1F1230J',
 'W1F13SRV',
 'W1F14XGD',
 'W1F15S4D',
 'W1F19BPT',
 'W1F1BFP5',
 'W1F1BS0H',

### Separating the failed devices into a separate data frame for easier viewing.

In [57]:
failed_df = df[df['device'].isin(failed_devices)]
failed_df.head()

,date,device,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9,failure
3496,15004,S1F023H2,204752808,0,0,1,19,497559,16,16,3,0
12198,15014,S1F023H2,203744280,0,0,1,19,509250,16,16,3,0
10688,15012,S1F023H2,156754936,0,0,1,19,506684,16,16,3,0
5819,15006,S1F023H2,10387472,0,0,1,19,499964,16,16,3,0
6868,15007,S1F023H2,30083248,0,0,1,19,501239,16,16,3,0


In [ ]:
failed_df.reset_index(inplace=True)

### Looking more closely at telemetry data.

In [46]:
df.aggregate(np.mean)

date          1.510622e+04
attribute1    1.223868e+08
attribute2    1.594848e+02
attribute3    9.940455e+00
attribute4    1.741120e+00
attribute5    1.422269e+01
attribute6    2.601729e+05
attribute7    2.925282e-01
attribute8    2.925282e-01
attribute9    1.245152e+01
failure       8.514467e-04
dtype: float64

In [47]:
df.query('failure == 1').aggregate(np.mean)

date          1.510791e+04
attribute1    1.271755e+08
attribute2    4.109434e+03
attribute3    3.905660e+00
attribute4    5.463208e+01
attribute5    1.546226e+01
attribute6    2.583035e+05
attribute7    3.062264e+01
attribute8    3.062264e+01
attribute9    2.308491e+01
failure       1.000000e+00
dtype: float64

In [48]:
df.query('failure==0').aggregate(np.mean)

date          1.510622e+04
attribute1    1.223827e+08
attribute2    1.561187e+02
attribute3    9.945598e+00
attribute4    1.696048e+00
attribute5    1.422164e+01
attribute6    2.601745e+05
attribute7    2.666817e-01
attribute8    2.666817e-01
attribute9    1.244246e+01
failure       0.000000e+00
dtype: float64

In [49]:
df.query("device == 'S1F0RRB1'")

,date,device,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9,failure
2561,15003,S1F0RRB1,26258330,64776,0,135,8,39267,56,56,1,0
1398,15002,S1F0RRB1,13307628,64776,0,49,8,39267,56,56,1,0
4885,15005,S1F0RRB1,48467332,64776,0,841,8,39267,56,56,1,1
235,15001,S1F0RRB1,5230888,2288,0,37,8,39267,24,24,1,0
3724,15004,S1F0RRB1,37985862,64776,0,763,8,39267,56,56,1,0


In [50]:
df.query("device == 'W1F0F6BN'")

,date,device,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9,failure
12517,15014,W1F0F6BN,160516016,0,0,0,12,241283,0,0,0,0
1720,15002,W1F0F6BN,132121976,0,0,0,12,226495,0,0,0,0
4046,15004,W1F0F6BN,176403712,0,0,0,12,228862,0,0,0,0
13233,15015,W1F0F6BN,180323664,0,0,0,12,242533,0,0,0,0
8765,15009,W1F0F6BN,49165752,0,0,0,12,235082,0,0,0,0
14663,15017,W1F0F6BN,148907576,0,0,16,12,243751,0,0,0,0
5207,15005,W1F0F6BN,200875200,0,0,0,12,230151,0,0,0,0
11786,15013,W1F0F6BN,139922104,0,0,0,12,239953,0,0,0,0
8009,15008,W1F0F6BN,22376168,0,0,0,12,234010,0,0,0,0
16091,15019,W1F0F6BN,140931864,0,0,28,12,243751,152,152,0,0


In [51]:
df.query("device == 'S1F0T2LA'")

,date,device,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9,failure
28053,15036,S1F0T2LA,227598776,424,0,1,10,272837,0,0,0,0
81764,15135,S1F0T2LA,37246592,424,0,1,10,277260,0,0,0,0
29476,15038,S1F0T2LA,138045016,424,0,1,10,272837,0,0,0,0
58175,15080,S1F0T2LA,188053208,424,0,1,10,272873,0,0,0,0
85341,15145,S1F0T2LA,121103736,424,0,1,10,279125,0,0,0,0
88130,15153,S1F0T2LA,80838576,424,0,1,10,279133,0,0,0,0
82125,15136,S1F0T2LA,11802416,424,0,1,10,277260,0,0,0,0
42666,15057,S1F0T2LA,207395200,424,0,1,10,272855,0,0,0,0
102732,15195,S1F0T2LA,74360912,440,0,1,10,285324,0,0,0,0
101707,15192,S1F0T2LA,13017992,440,0,1,10,281401,0,0,0,0


Telemetry data ends when device fails and some attributes seem to be constant for each device.  


In [52]:
df.groupby('device')['attribute5'].mean()

device
S1F01085     6.000000
S1F013BB     5.000000
S1F0166B     6.000000
S1F01E6Y    12.000000
S1F01JE0     6.000000
              ...    
Z1F1VMZB     5.000000
Z1F1VQFY     6.328000
Z1F26YZB     1.000000
Z1F282ZV     1.000000
Z1F2PBHX     4.927711
Name: attribute5, Length: 1168, dtype: float64

In [58]:
df.query('device == "Z1F2PBHX"').head(50)

,date,device,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9,failure
47894,15064,Z1F2PBHX,227647808,0,0,0,5,161010,0,0,0,0
27167,15034,Z1F2PBHX,147243080,0,0,0,5,157414,0,0,0,0
37131,15048,Z1F2PBHX,102391480,0,0,0,5,157424,0,0,0,0
47219,15063,Z1F2PBHX,174441664,0,0,0,5,161010,0,0,0,0
57354,15078,Z1F2PBHX,193182840,0,0,0,5,161015,0,0,0,0
32862,15042,Z1F2PBHX,100943576,0,0,0,5,157415,0,0,0,0
5811,15005,Z1F2PBHX,205455544,0,0,0,4,149672,0,0,0,0
58026,15079,Z1F2PBHX,4344360,0,0,0,5,161016,0,0,0,0
3488,15003,Z1F2PBHX,188059992,0,0,0,4,148008,0,0,0,0
24319,15030,Z1F2PBHX,242857056,0,0,0,5,157410,0,0,0,0


Mean value of `attribute5` for devices that failed.

In [54]:
df[df['device'].isin(failed_devices)].groupby(
    'device')['attribute5'].mean().sort_values().unique()

array([ 2.90829694,  4.        ,  5.73076923,  5.85714286,  6.        ,
        6.328     ,  6.33333333,  6.4       ,  6.53846154,  6.89447236,
        7.        ,  7.01526718,  7.04964539,  7.5       ,  7.57142857,
        7.74698795,  7.85714286,  7.89119171,  7.90232558,  7.92446043,
        8.        ,  8.17283951,  8.72368421,  8.89756098,  8.90869565,
        9.        ,  9.83464567,  9.87573964, 10.        , 10.28813559,
       10.45454545, 11.        , 11.19230769, 11.53333333, 11.85810811,
       12.        , 12.13793103, 12.72368421, 13.        , 13.01666667,
       13.93814433, 14.        , 14.65263158, 14.66666667, 14.78672986,
       15.71428571, 16.        , 16.856     , 19.        , 23.        ,
       24.        , 25.        , 30.        , 30.5       , 33.775     ,
       35.325     , 35.60869565, 35.72727273, 36.74285714, 40.1025641 ,
       58.        , 64.        , 65.        , 90.        , 91.        ])

Mean value of `attribute5` for devices that didn't fail.  

In [55]:
df[~df['device'].isin(failed_devices)].groupby(
    'device')['attribute5'].mean().sort_values().unique()

array([ 1.        ,  2.        ,  2.97087379,  3.        ,  3.03289474,
        3.38135593,  4.        ,  4.38135593,  4.4516129 ,  4.5       ,
        4.81443299,  4.92771084,  5.        ,  5.22222222,  5.38135593,
        5.5       ,  5.6       ,  5.7635468 ,  5.92579505,  5.9384058 ,
        6.        ,  6.03289474,  6.11111111,  6.11458333,  6.218107  ,
        6.40707965,  6.5       ,  6.8452381 ,  6.90721649,  6.91864407,
        6.93617021,  6.95510204,  6.97029703,  6.97627119,  7.        ,
        7.03289474,  7.40707965,  7.41818182,  7.43654822,  7.5       ,
        7.66666667,  7.69230769,  7.89166667,  7.90721649,  7.91428571,
        7.92250923,  7.92307692,  7.92579505,  7.97087379,  7.97849462,
        8.        ,  8.03289474,  8.05357143,  8.05555556,  8.10714286,
        8.17777778,  8.20205479,  8.31147541,  8.40707965,  8.42718447,
        8.49411765,  8.5       ,  8.51612903,  8.75342466,  8.85365854,
        8.9047619 ,  8.92070485,  8.92579505,  8.99056604,  9.  

`attribute5` is mostly the same for each device and there doesn't seem to be any meaningful difference 
between failed and not-failed devices.

### Telemetry progression of the failed devices

In [67]:
failed_df.sort_values(by='date', inplace=True)
failed_grid = qgrid.QGridWidget(df=failed_df, show_toolbar=True)
failed_grid

C:\Users\flake\Anaconda3\envs\ga_dsi\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…